In [34]:
import geopandas as gpd
import networkx as nx
from osmnx import features

from cityseer.tools import io, graphs
from cityseer.metrics import networks, layers

In [35]:
from pathlib import Path

repo_path = Path.cwd()
if str(repo_path).endswith("demos"):
    repo_path = Path.cwd() / ".."
if not str(repo_path.resolve()).endswith("cityseer-api"):
    raise ValueError("Please check your notebook working directory relative to your project and data paths.")

bubenec_path = Path(repo_path / "temp/bubenec.gpkg")
print("data path:", bubenec_path)
print("path exists:", bubenec_path.exists())

df_streets: gpd.GeoDataFrame = gpd.read_file(bubenec_path, layer="streets")  # type: ignore

data path: /Users/gareth/dev/benchmark-urbanism/cityseer-api/demos/../temp/bubenec.gpkg
path exists: True


In [36]:
data_gdf = features.features_from_point((50.1029248, 14.4029967), tags={"amenity": ["pub", "restaurant"]}, dist=400)
data_gdf.to_crs(3857, inplace=True)
data_gdf = data_gdf.loc["node"]
data_gdf = data_gdf.reset_index(level=0, drop=True)
data_gdf.index = data_gdf.index.astype(str)
data_gdf = data_gdf[["amenity", "geometry"]]
print(data_gdf.head())

      amenity                         geometry
0  restaurant  POINT (1602992.986 6463522.177)
1  restaurant  POINT (1602903.753 6463784.958)
2  restaurant  POINT (1603489.527 6463552.391)
3  restaurant  POINT (1602809.665 6464167.222)
4  restaurant  POINT (1603027.863 6464060.660)


In [37]:
def nx_from_momepy(
    gdf_network: gpd.GeoDataFrame,  # type: ignore
) -> nx.MultiGraph:
    """
    Converts a `momepy` LineString GeoDataFrame to a `cityseer` compatible `networkX` `MultiGraph`.

    The `momepy` GeoDataFrame should be provided in the "primal" form, where edges represent true

    Parameters
    ----------
    gdf_network : GeoDataFrame
        A `momepy` LineString GeoDataFrame.

    Returns
    -------
    MultiGraph
        A `cityseer` compatible `networkX` graph with `x` and `y` node attributes and `geom` edge attributes.

    """
    gdf_network: gpd.GeoDataFrame = gdf_network.copy()  # type: ignore
    if not gdf_network.crs.is_projected:
        raise ValueError("The GeoDataframe CRS must be projected, i.e. not geographic.")
    g_multi = nx.MultiGraph()
    g_multi.graph["crs"] = gdf_network.crs

    def _node_key(node_coords: list[float]):
        x = round(node_coords[0], 3)
        y = round(node_coords[1], 3)
        z = round(node_coords[2], 3) if len(node_coords) == 3 else "NaN"
        return f"x{x}-y{y}-z{z}"

    geom_key = gdf_network.geometry.name
    for edge_idx, edge_row in gdf_network.iterrows():
        # generate start and ending nodes
        edge_geom = edge_row[geom_key]
        coords_a = edge_geom.coords[0]
        node_key_a = _node_key(coords_a)
        if node_key_a not in g_multi:
            g_multi.add_node(node_key_a, x=coords_a[0], y=coords_a[1])
        coords_b = edge_geom.coords[-1]
        node_key_b = _node_key(coords_b)
        if node_key_b not in g_multi:
            g_multi.add_node(node_key_b, x=coords_b[0], y=coords_b[1])
        g_multi.add_edge(node_key_a, node_key_b, momepy_edge_idx=edge_idx, geom=edge_geom)

    return g_multi

In [38]:
nx_momepy = io.nx_from_momepy(df_streets)
# optionally use cityseer to clean network
# if using primal network
nodes_gdf, edges_gdf, network_structure = graphs.network_structure_from_nx(nx_momepy, crs=nx_momepy.graph["crs"])

INFO:cityseer.tools.graphs:Preparing node and edge arrays from networkX graph.
100%|██████████| 29/29 [00:00<00:00, 1910.90it/s]


In [39]:
nodes_gdf.head()

,ns_node_idx,x,y,live,geom
x1603585.64-y6464428.774-zNaN,0,1.603586e+06,6.464429e+06,True,POINT (1603585.640 6464428.774)
x1603413.206-y6464228.73-zNaN,1,1.603413e+06,6.464229e+06,True,POINT (1603413.206 6464228.730)
x1603268.502-y6464060.781-zNaN,2,1.603269e+06,6.464061e+06,True,POINT (1603268.502 6464060.781)
x1603363.558-y6464031.885-zNaN,3,1.603364e+06,6.464032e+06,True,POINT (1603363.558 6464031.885)
x1603607.303-y6464181.853-zNaN,4,1.603607e+06,6.464182e+06,True,POINT (1603607.303 6464181.853)


In [47]:
edges_gdf.head(3)

,ns_edge_idx,start_ns_node_idx,end_ns_node_idx,edge_idx,nx_start_node_key,nx_end_node_key,length,angle_sum,imp_factor,in_bearing,out_bearing,total_bearing,geom
x1603585.64-y6464428.774-zNaN-x1603413.206-y6464228.73-zNaN,0,0,1,0,x1603585.64-y6464428.774-zNaN,x1603413.206-y6464228.73-zNaN,264.103950,0.000000,1,-130.760729,-130.760729,-130.760729,"LINESTRING (1603585.640 6464428.774, 1603413.2..."
x1603585.64-y6464428.774-zNaN-x1603561.74-y6464494.467-zNaN,1,0,8,0,x1603585.64-y6464428.774-zNaN,x1603561.74-y6464494.467-zNaN,70.020202,8.483583,1,111.541453,103.057871,109.992304,"LINESTRING (1603561.740 6464494.467, 1603564.6..."
x1603585.64-y6464428.774-zNaN-x1603650.45-y6464368.601-zNaN,2,0,6,0,x1603585.64-y6464428.774-zNaN,x1603650.45-y6464368.601-zNaN,88.924305,19.246836,1,-42.077556,-55.845266,-42.875276,"LINESTRING (1603585.640 6464428.774, 1603603.0..."


In [46]:
data_gdf["amenity"].head(10)

0    restaurant
1    restaurant
2    restaurant
3    restaurant
4    restaurant
5    restaurant
6           pub
7    restaurant
8           pub
9    restaurant
Name: amenity, dtype: object

In [44]:
nodes_gdf.head()

,ns_node_idx,x,y,live,geom,cc_metric_node_beta_250,cc_metric_node_beta_500,cc_metric_node_beta_1000,cc_metric_node_cycles_250,cc_metric_node_cycles_500,...,cc_metric_node_betweenness_1000,cc_metric_node_betweenness_beta_250,cc_metric_node_betweenness_beta_500,cc_metric_node_betweenness_beta_1000,cc_metric_restaurant_100_non_weighted,cc_metric_restaurant_100_weighted,cc_metric_restaurant_200_non_weighted,cc_metric_restaurant_200_weighted,cc_metric_restaurant_400_non_weighted,cc_metric_restaurant_400_weighted
x1603585.64-y6464428.774-zNaN,0,1.603586e+06,6.464429e+06,True,POINT (1603585.640 6464428.774),0.716697,2.050036,4.991603,0.0,3.0,...,77.0,0.105017,1.056299,6.454368,0.0,0.000000,0.0,0.000000,1.0,0.043205
x1603413.206-y6464228.73-zNaN,1,1.603413e+06,6.464229e+06,True,POINT (1603413.206 6464228.730),0.121535,1.309901,5.305615,0.0,4.0,...,102.0,0.000000,0.089877,5.084879,1.0,0.134929,1.0,0.367327,2.0,0.660077
x1603268.502-y6464060.781-zNaN,2,1.603269e+06,6.464061e+06,True,POINT (1603268.502 6464060.781),0.434456,1.921538,5.837987,0.0,3.0,...,41.0,0.036077,0.693734,4.369751,1.0,0.309082,1.0,0.555951,5.0,0.895136
x1603363.558-y6464031.885-zNaN,3,1.603364e+06,6.464032e+06,True,POINT (1603363.558 6464031.885),0.390683,1.850008,6.036365,1.0,5.0,...,87.0,0.000000,0.676959,6.886633,0.0,0.000000,1.0,0.075615,2.0,0.354570
x1603607.303-y6464181.853-zNaN,4,1.603607e+06,6.464182e+06,True,POINT (1603607.303 6464181.853),0.284579,1.452106,4.887573,1.0,3.0,...,45.0,0.000000,0.263284,3.326667,0.0,0.000000,0.0,0.000000,1.0,0.082232


In [43]:
# compute metrics
nodes_gdf = networks.node_centrality_shortest(network_structure, nodes_gdf, [250, 500, 1000])
nodes_gdf, data_gdf = layers.compute_accessibilities(
    data_gdf,
    landuse_column_label="amenity",
    accessibility_keys=["restaurant"],
    nodes_gdf=nodes_gdf,
    network_structure=network_structure,
    distances=[100, 200, 400],
)
nodes_gdf.head()

  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [00:00<00:00, 3349.44it/s]
INFO:cityseer.metrics.layers:Computing land-use accessibility for: restaurant
100%|██████████| 29/29 [00:01<00:00, 28.94it/s]


,ns_node_idx,x,y,live,geom,cc_metric_node_beta_250,cc_metric_node_beta_500,cc_metric_node_beta_1000,cc_metric_node_cycles_250,cc_metric_node_cycles_500,...,cc_metric_node_betweenness_1000,cc_metric_node_betweenness_beta_250,cc_metric_node_betweenness_beta_500,cc_metric_node_betweenness_beta_1000,cc_metric_restaurant_100_non_weighted,cc_metric_restaurant_100_weighted,cc_metric_restaurant_200_non_weighted,cc_metric_restaurant_200_weighted,cc_metric_restaurant_400_non_weighted,cc_metric_restaurant_400_weighted
x1603585.64-y6464428.774-zNaN,0,1.603586e+06,6.464429e+06,True,POINT (1603585.640 6464428.774),0.716697,2.050036,4.991603,0.0,3.0,...,77.0,0.105017,1.056299,6.454368,0.0,0.000000,0.0,0.000000,1.0,0.043205
x1603413.206-y6464228.73-zNaN,1,1.603413e+06,6.464229e+06,True,POINT (1603413.206 6464228.730),0.121535,1.309901,5.305615,0.0,4.0,...,102.0,0.000000,0.089877,5.084879,1.0,0.134929,1.0,0.367327,2.0,0.660077
x1603268.502-y6464060.781-zNaN,2,1.603269e+06,6.464061e+06,True,POINT (1603268.502 6464060.781),0.434456,1.921538,5.837987,0.0,3.0,...,41.0,0.036077,0.693734,4.369751,1.0,0.309082,1.0,0.555951,5.0,0.895136
x1603363.558-y6464031.885-zNaN,3,1.603364e+06,6.464032e+06,True,POINT (1603363.558 6464031.885),0.390683,1.850008,6.036365,1.0,5.0,...,87.0,0.000000,0.676959,6.886633,0.0,0.000000,1.0,0.075615,2.0,0.354570
x1603607.303-y6464181.853-zNaN,4,1.603607e+06,6.464182e+06,True,POINT (1603607.303 6464181.853),0.284579,1.452106,4.887573,1.0,3.0,...,45.0,0.000000,0.263284,3.326667,0.0,0.000000,0.0,0.000000,1.0,0.082232


In [50]:
# if using dual network
nx_dual = graphs.nx_to_dual(nx_momepy)
nodes_gdf_dual, _edges_gdf_dual, network_structure_dual = graphs.network_structure_from_nx(
    nx_dual, nx_momepy.graph["crs"]
)
nodes_gdf_dual = networks.node_centrality_simplest(network_structure_dual, nodes_gdf_dual, [500])
nodes_gdf_dual.head()

INFO:cityseer.tools.graphs:Converting graph to dual.
100%|██████████| 35/35 [00:00<00:00, 229.59it/s]
INFO:cityseer.tools.graphs:Preparing node and edge arrays from networkX graph.
100%|██████████| 35/35 [00:01<00:00, 34.92it/s]


,ns_node_idx,x,y,live,geom,cc_metric_node_harmonic_simplest_500,cc_metric_node_betweenness_simplest_500
x1603413.206-y6464228.73-zNaN_x1603585.64-y6464428.774-zNaN,0,1.603499e+06,6.464329e+06,True,POINT (1603499.423 6464328.752),11.059296,9.0
x1603561.74-y6464494.467-zNaN_x1603585.64-y6464428.774-zNaN,1,1.603573e+06,6.464461e+06,True,POINT (1603572.785 6464461.339),7.831646,10.0
x1603585.64-y6464428.774-zNaN_x1603650.45-y6464368.601-zNaN,2,1.603619e+06,6.464400e+06,True,POINT (1603619.295 6464399.737),7.865049,14.0
x1603413.206-y6464228.73-zNaN_x1603607.303-y6464181.853-zNaN,3,1.603510e+06,6.464204e+06,True,POINT (1603510.094 6464204.494),13.298367,8.0
x1603363.558-y6464031.885-zNaN_x1603413.206-y6464228.73-zNaN,4,1.603388e+06,6.464130e+06,True,POINT (1603388.005 6464130.401),14.422626,10.0
